# AIMMS Chap.19
**2024.01.24**

In [2]:
# Import
import math
import numpy as np
import pandas as pd

from gurobipy import Model, GRB, quicksum

In [3]:
# Dataの定義
IncomeData = np.array([
    [20, 10000, 3, "Commerce", 0],
    [30, 15500, 2, "Agriculture", 1000], 
    [25, 20000, 5, "Agriculture", 1000], 
    [18, 25000, 4, "Commerce", 3000],
    [32, 15000, 3, "Commerce", 500]
    ])

PopulationData = np.array([
    [25, 15000, 4, 40, 12, "M", 2],
    [30, 15000, 2, 25, 16, "M", 0],
    [18, 20000, 1, 30, 18, "F", 0],
    [27, 25000, 2, 35, 16, "F", 1],
    [25, 20000, 4, 25, 12, "M", 1],
    ])

In [4]:
pd.DataFrame(IncomeData, columns=["No. of Families", "Gross Family Income", "No. of Familiy Members", "Source of Income", "Interest Income"])

,No. of Families,Gross Family Income,No. of Familiy Members,Source of Income,Interest Income
0,20,10000,3,Commerce,0
1,30,15500,2,Agriculture,1000
2,25,20000,5,Agriculture,1000
3,18,25000,4,Commerce,3000
4,32,15000,3,Commerce,500


In [5]:
pd.DataFrame(PopulationData, columns=["No. of Families", "Gross Family Income", "No. of Familiy Members", "Head of Household(Age)", "Head of Household(Education)", "Head of Household(Sex)", "No. of Familiy Members Under 18"])

,No. of Families,Gross Family Income,No. of Familiy Members,Head of Household(Age),Head of Household(Education),Head of Household(Sex),No. of Familiy Members Under 18
0,25,15000,4,40,12,M,2
1,30,15000,2,25,16,M,0
2,18,20000,1,30,18,F,0
3,27,25000,2,35,16,F,1
4,25,20000,4,25,12,M,1


In [6]:
model = Model()

Restricted license - for non-production use only - expires 2024-10-28


In [7]:
# 添字
"""
i : supply nodes 
j : demand nodes 
"""
# パラメータ
## No. of Families
N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

## Gross Family Income
G_i = IncomeData[:, 1].astype(int)
G_j = PopulationData[:, 1].astype(int)

## No. of Family Members
M_i = IncomeData[:, 2].astype(int)
M_j = PopulationData[:, 2].astype(int)

## 分散
s_G = np.var(np.concatenate([G_i, G_j])) # Gross
s_M = np.var(np.concatenate([M_i, M_j])) # Member

# 変数
x_ij = {(i, j): model.addVar(vtype=GRB.INTEGER) for i in range(5) for j in range(5)}
d_ij = {(i, j): model.addVar(vtype=GRB.CONTINUOUS) for i in range(5) for j in range(5)}


In [8]:
print(G_i)
print(G_j)
print(N_i)
print(N_j)

[10000 15500 20000 25000 15000]
[15000 15000 20000 25000 20000]
[20 30 25 18 32]
[25 30 18 27 25]


In [9]:
# 制約
## Σx_ij = N_i
model.addConstrs((quicksum(x_ij[i, j] for j in range(5)) - N_i[i] == 0 for i in range(5)))

## Σx_ij = N_j
model.addConstrs((quicksum(x_ij[i, j] for i in range(5)) - N_j[j] == 0 for j in range(5)))

## x_ij >= 0
model.addConstrs((x_ij[i, j] >= 0 for j in range(5) for i in range(5)))

# d
model.addConstrs(d_ij[i, j] == math.sqrt((G_i[i]-G_j[j])**2/s_G+(M_i[i]-M_j[j])**2/s_M) for i in range(5) for j in range(5))

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,


In [10]:
model.setObjective(quicksum(d_ij[i,j]*x_ij[i,j] for i in range(5) for j in range(5)),GRB.MINIMIZE); #目的関数の追加

model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 60 rows, 50 columns and 100 nonzeros
Model fingerprint: 0xf6de555c
Model has 25 quadratic objective terms
Variable types: 25 continuous, 25 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 3e+01]
Found heuristic solution: objective 252.9675974
Presolve removed 50 rows and 25 columns
Presolve time: 0.01s
Presolved: 10 rows, 25 columns, 50 nonzeros
Variable types: 0 continuous, 25 integer (0 binary)

Root relaxation: objective 1.442615e+02, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0    

In [11]:
#最適値の出力
print("Opt.value=",model.objval)

Opt.value= 144.26148655218032


In [12]:
for i in range(5): #最適解の出力
    for j in range(5):
        print("x_ij[",i,"][",j,"]=",x_ij[i,j].X)

x_ij[ 0 ][ 0 ]= 20.0
x_ij[ 0 ][ 1 ]= -0.0
x_ij[ 0 ][ 2 ]= -0.0
x_ij[ 0 ][ 3 ]= -0.0
x_ij[ 0 ][ 4 ]= -0.0
x_ij[ 1 ][ 0 ]= -0.0
x_ij[ 1 ][ 1 ]= 30.0
x_ij[ 1 ][ 2 ]= -0.0
x_ij[ 1 ][ 3 ]= -0.0
x_ij[ 1 ][ 4 ]= -0.0
x_ij[ 2 ][ 0 ]= -0.0
x_ij[ 2 ][ 1 ]= -0.0
x_ij[ 2 ][ 2 ]= -0.0
x_ij[ 2 ][ 3 ]= -0.0
x_ij[ 2 ][ 4 ]= 25.0
x_ij[ 3 ][ 0 ]= -0.0
x_ij[ 3 ][ 1 ]= -0.0
x_ij[ 3 ][ 2 ]= -0.0
x_ij[ 3 ][ 3 ]= 18.0
x_ij[ 3 ][ 4 ]= 0.0
x_ij[ 4 ][ 0 ]= 5.0
x_ij[ 4 ][ 1 ]= -0.0
x_ij[ 4 ][ 2 ]= 18.0
x_ij[ 4 ][ 3 ]= 9.0
x_ij[ 4 ][ 4 ]= -0.0


In [13]:
for i in range(5): #それぞれの距離の出力
    for j in range(5):
        print("d_ij[",i,"][",j,"]=",d_ij[i,j].X)

d_ij[ 0 ][ 0 ]= 1.3858946059547992
d_ij[ 0 ][ 1 ]= 1.3858946059547992
d_ij[ 0 ][ 2 ]= 2.7717892119095984
d_ij[ 0 ][ 3 ]= 3.401771452500264
d_ij[ 0 ][ 4 ]= 2.3537115992409285
d_ij[ 1 ][ 0 ]= 1.693873383281096
d_ij[ 1 ][ 1 ]= 0.1098370677198228
d_ij[ 1 ][ 2 ]= 1.3005708021304023
d_ij[ 1 ][ 3 ]= 2.0869042866766327
d_ij[ 1 ][ 4 ]= 1.9581474801994003
d_ij[ 2 ][ 0 ]= 1.3858946059547992
d_ij[ 2 ][ 1 ]= 2.763148489151519
d_ij[ 2 ][ 2 ]= 3.3806170189140663
d_ij[ 2 ][ 3 ]= 2.763148489151519
d_ij[ 2 ][ 4 ]= 0.8451542547285166
d_ij[ 3 ][ 0 ]= 2.1967413543964556
d_ij[ 3 ][ 1 ]= 2.7717892119095984
d_ij[ 3 ][ 2 ]= 2.763148489151519
d_ij[ 3 ][ 3 ]= 1.6903085094570331
d_ij[ 3 ][ 4 ]= 1.0983706771982278
d_ij[ 4 ][ 0 ]= 0.8451542547285166
d_ij[ 4 ][ 1 ]= 0.8451542547285166
d_ij[ 4 ][ 2 ]= 2.0158276220132887
d_ij[ 4 ][ 3 ]= 2.3537115992409285
d_ij[ 4 ][ 4 ]= 1.3858946059547992


In [14]:
MergeData = []
for i in range(5):
    for j in range(5):
        if (x_ij[i, j].X > 0):
            MergeData.append(
                np.concatenate([
                    np.array([x_ij[i, j].X, (G_i[i] + G_j[j]) / 2, max(M_i[i], M_j[j])]).astype(str),
                    IncomeData[i][3:].astype(str),
                    PopulationData[j][3:].astype(str)
                ])
            )
pd.DataFrame(MergeData, columns=["No. of Families", "Gross Family Income", "No. of Familiy Members", "Source of Income", "Interest Income", "Head of Household(Age)", "Head of Household(Education)", "Head of Household(Sex)", "No. of Familiy Members Under 18"])

,No. of Families,Gross Family Income,No. of Familiy Members,Source of Income,Interest Income,Head of Household(Age),Head of Household(Education),Head of Household(Sex),No. of Familiy Members Under 18
0,20.0,12500.0,4.0,Commerce,0,40,12,M,2
1,30.0,15250.0,2.0,Agriculture,1000,25,16,M,0
2,25.0,20000.0,5.0,Agriculture,1000,25,12,M,1
3,18.0,25000.0,4.0,Commerce,3000,35,16,F,1
4,5.0,15000.0,4.0,Commerce,500,40,12,M,2
5,18.0,17500.0,3.0,Commerce,500,30,18,F,0
6,9.0,20000.0,3.0,Commerce,500,35,16,F,1


## Exercize2

In [15]:
sortedRankI = [0, 4, 1, 2, 3] # IncomeDataの行の並び替え
sortedRankP = [0, 1, 2, 4, 3] # PopulationDataの行の並び替え

In [16]:
# 添字
"""
i : supply nodes 
j : demand nodes 
"""
# パラメータ
## No. of Families
N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

## Gross Family Income
G_i = IncomeData[:, 1].astype(int)
G_j = PopulationData[:, 1].astype(int)

## No. of Family Members
M_i = IncomeData[:, 2].astype(int)
M_j = PopulationData[:, 2].astype(int)

## 分散
s_G = np.var(np.concatenate([G_i, G_j])) # Gross
s_M = np.var(np.concatenate([M_i, M_j])) # Member

# 変数

x_ij = {(i, j): model.addVar(vtype=GRB.INTEGER) for i in range(5) for j in range(5)}
d_ij = {(i, j): model.addVar(vtype=GRB.CONTINUOUS) for i in range(5) for j in range(5)}

In [ ]:
# Initialization (1) N_i, N_jの全ての要素からNを引く

sortedRankI = [0, 4, 1, 2, 3] # IncomeDataの行の並び替え (昇順)
sortedRankP = [0, 1, 2, 4, 3] # PopulationDataの行の並び替え (昇順)

N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

I = iter(sortedRankI) 
J = iter(sortedRankP)
i_star = next(I)
j_star = next(J) 
S = []
x_ij = [[0 for _ in range(len(sortedRankI))] for _ in range(len(sortedRankP))]

try:
    while True:
        print(f"i_star: {i_star}")
        print(f"j_star: {j_star}")
        while N_i[i_star]<=0:
            i_star = next(I)
        while N_j[j_star]<=0:
            j_star = next(J)
        N_star = min(N_i[i_star], N_j[j_star])
        x_ij[i_star][j_star] = N_star
        N_i -= N_star
        N_j -= N_star
        print(i_star, j_star, N_star)
        print(N_i)
        print(N_j)
        S.append((i_star, j_star))
except StopIteration: 
    print("Finished.")

print(" ============= 表示 ============= ")
print(S)
for i in range(len(sortedRankI)): 
    for j in range(len(sortedRankP)):
        print("x_ij[",i,"][",j,"]=",x_ij[i][j])

In [19]:
# Initialization (2) N_i[i_star], N_j[j_star]からNを引く

sortedRankI = [0, 4, 1, 2, 3] # IncomeDataの行の並び替え (昇順)
sortedRankP = [0, 1, 2, 4, 3] # PopulationDataの行の並び替え (昇順)

N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

I = iter(sortedRankI) 
J = iter(sortedRankP)
i_star = next(I)
j_star = next(J) 
S = []
x_ij = [[0 for _ in range(len(sortedRankI))] for _ in range(len(sortedRankP))]

try:
    while True:
        print(f"i_star: {i_star}")
        print(f"j_star: {j_star}")
        while N_i[i_star]<=0:
            i_star = next(I)
        while N_j[j_star]<=0:
            j_star = next(J)
        N_star = min(N_i[i_star], N_j[j_star])
        x_ij[i_star][j_star] = N_star
        N_i[i_star] -= N_star
        N_j[j_star] -= N_star
        print(i_star, j_star, N_star)
        print(N_i)
        print(N_j)
        S.append((i_star, j_star))
except StopIteration: 
    print("Finished.")


i_star: 0
j_star: 0
0 0 20
[ 0 30 25 18 32]
[ 5 30 18 27 25]
i_star: 0
j_star: 0
4 0 5
[ 0 30 25 18 27]
[ 0 30 18 27 25]
i_star: 4
j_star: 0
4 1 27
[ 0 30 25 18  0]
[ 0  3 18 27 25]
i_star: 4
j_star: 1
1 1 3
[ 0 27 25 18  0]
[ 0  0 18 27 25]
i_star: 1
j_star: 1
1 2 18
[ 0  9 25 18  0]
[ 0  0  0 27 25]
i_star: 1
j_star: 2
1 4 9
[ 0  0 25 18  0]
[ 0  0  0 27 16]
i_star: 1
j_star: 4
2 4 16
[ 0  0  9 18  0]
[ 0  0  0 27  0]
i_star: 2
j_star: 4
2 3 9
[ 0  0  0 18  0]
[ 0  0  0 18  0]
i_star: 2
j_star: 3
3 3 18
[0 0 0 0 0]
[0 0 0 0 0]
i_star: 3
j_star: 3
Finished.


In [20]:
print(" ============= 表示 ============= ")
print(S)
for i in range(len(sortedRankI)): 
    for j in range(len(sortedRankP)):
        print("x_ij[",i,"][",j,"]=",x_ij[i][j])

 ============= 表示 ============= 
[(0, 0), (4, 0), (4, 1), (1, 1), (1, 2), (1, 4), (2, 4), (2, 3), (3, 3)]
x_ij[ 0 ][ 0 ]= 20
x_ij[ 0 ][ 1 ]= 0
x_ij[ 0 ][ 2 ]= 0
x_ij[ 0 ][ 3 ]= 0
x_ij[ 0 ][ 4 ]= 0
x_ij[ 1 ][ 0 ]= 0
x_ij[ 1 ][ 1 ]= 3
x_ij[ 1 ][ 2 ]= 18
x_ij[ 1 ][ 3 ]= 0
x_ij[ 1 ][ 4 ]= 9
x_ij[ 2 ][ 0 ]= 0
x_ij[ 2 ][ 1 ]= 0
x_ij[ 2 ][ 2 ]= 0
x_ij[ 2 ][ 3 ]= 9
x_ij[ 2 ][ 4 ]= 16
x_ij[ 3 ][ 0 ]= 0
x_ij[ 3 ][ 1 ]= 0
x_ij[ 3 ][ 2 ]= 0
x_ij[ 3 ][ 3 ]= 18
x_ij[ 3 ][ 4 ]= 0
x_ij[ 4 ][ 0 ]= 5
x_ij[ 4 ][ 1 ]= 27
x_ij[ 4 ][ 2 ]= 0
x_ij[ 4 ][ 3 ]= 0
x_ij[ 4 ][ 4 ]= 0


### 初期化↑で合ってる分からないけどこんな感じな気がする。
### その後のフローチャートが全然ピンとこない。

In [9]:
# calc_Dij
def calc_Dij(i, j, G_i, G_j, M_i, M_j):
    s_G = np.var(np.concatenate([G_i, G_j])) # Gross
    s_M = np.var(np.concatenate([M_i, M_j])) # Member
    return  math.sqrt((G_i[j]-G_j[j])**2/s_G+(M_i[i]-M_j[j])**2/s_M)

In [12]:
print(calc_Dij(0, 0, [10000, 15000, 15500, 20000, 25000], [15000, 15000, 20000, 20000, 25000], [3, 3, 2, 5, 4], [4, 2, 1, 4, 2]))
print(calc_Dij(4, 0, [10000, 15000, 15500, 20000, 25000], [15000, 15000, 20000, 20000, 25000], [3, 3, 2, 5, 4], [4, 2, 1, 4, 2]))
print(calc_Dij(4, 1, [10000, 15000, 15500, 20000, 25000], [15000, 15000, 20000, 20000, 25000], [3, 3, 2, 5, 4], [4, 2, 1, 4, 2]))

1.3858946059547992
1.0983706771982278
1.6903085094570331


In [149]:
## No. of Families
N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

## Gross Family Income
G_i = IncomeData[:, 1].astype(int)
G_j = PopulationData[:, 1].astype(int)
print(G_i)
print(G_j)

sorted_indices_i = sorted(range(len(G_i)), key=lambda k: G_i[k])
sorted_indices_j = sorted(range(len(G_j)), key=lambda k: G_j[k])
sorted_indices_i = [3,2,1,4,0]
sorted_indices_j = [3,4,2,1,0]
print(sorted_indices_i)
print(sorted_indices_j)

S = set()
x_ij_star_array = []


i_star = next((index for index, value in enumerate(N_i) if value > 0), None)
j_star = next((index for index, value in enumerate(N_j) if value > 0), None)
print(i_star)
print(j_star)

while i_star is not None and j_star is not None:
    print(i_star)
    N_star = min(N_i[i_star], N_j[j_star])
    x_ij_star = N_star
    N_i[i_star] -= N_star
    N_j[j_star] -= N_star
    S.add((i_star, j_star))
    x_ij_star_array.append(x_ij_star)
    i_star = next((index for index, value in enumerate(N_i) if value > 0), None)
    j_star = next((index for index, value in enumerate(N_j) if value > 0), None)


print(S)
print(x_ij_star_array)

[10000 15500 20000 25000 15000]
[15000 15000 20000 25000 20000]
[3, 2, 1, 4, 0]
[3, 4, 2, 1, 0]
0
0
0
1
1
2
2
2
3
4
4
{(4, 4), (2, 1), (0, 0), (4, 3), (1, 1), (2, 3), (3, 3), (2, 2), (1, 0)}
[20, 5, 25, 5, 18, 2, 18, 7, 25]


In [197]:
## No. of Families
N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

## Gross Family Income
G_i = IncomeData[:, 1].astype(int)
G_j = PopulationData[:, 1].astype(int)
S = [(2, 4), (3, 2), (1, 1), (0, 0), (4, 0), (4, 3)]
# [25, 18, 30, 20, 5, 27]

x_ij = {(i, j): model.addVar(vtype=GRB.INTEGER) for i in range(5) for j in range(5)}
print(x_ij)
# 制約
## Σx_ij(S) = N_i
model.addConstrs((quicksum(x_ij[i, j] for j in [e[1] for e in S]) - N_i[i] == 0 for i in [e[0] for e in S]))

## Σx_ij = N_j
model.addConstrs((quicksum(x_ij[i, j] for i in [e[0] for e in S]) - N_j[j] == 0 for j in [e[1] for e in S]))

## x_ij >= 0
model.addConstrs((x_ij[i, j] >= 0 for j in [e[1] for e in S] for i in [e[0] for e in S]))

# d
model.addConstrs(d_ij[i, j] == math.sqrt((G_i[i]-G_j[j])**2/s_G+(M_i[i]-M_j[j])**2/s_M) for i in [e[0] for e in S] for j in [e[1] for e in S])
## No. of Families
# N_i = IncomeData[:, 0].astype(int)
# N_j = PopulationData[:, 0].astype(int)
# i_star = next((index for index, value in enumerate(N_i) if value > 0), None)
# j_star = next((index for index, value in enumerate(N_j) if value > 0), None)
model.update()

{(0, 0): <gurobi.Var *Awaiting Model Update*>, (0, 1): <gurobi.Var *Awaiting Model Update*>, (0, 2): <gurobi.Var *Awaiting Model Update*>, (0, 3): <gurobi.Var *Awaiting Model Update*>, (0, 4): <gurobi.Var *Awaiting Model Update*>, (1, 0): <gurobi.Var *Awaiting Model Update*>, (1, 1): <gurobi.Var *Awaiting Model Update*>, (1, 2): <gurobi.Var *Awaiting Model Update*>, (1, 3): <gurobi.Var *Awaiting Model Update*>, (1, 4): <gurobi.Var *Awaiting Model Update*>, (2, 0): <gurobi.Var *Awaiting Model Update*>, (2, 1): <gurobi.Var *Awaiting Model Update*>, (2, 2): <gurobi.Var *Awaiting Model Update*>, (2, 3): <gurobi.Var *Awaiting Model Update*>, (2, 4): <gurobi.Var *Awaiting Model Update*>, (3, 0): <gurobi.Var *Awaiting Model Update*>, (3, 1): <gurobi.Var *Awaiting Model Update*>, (3, 2): <gurobi.Var *Awaiting Model Update*>, (3, 3): <gurobi.Var *Awaiting Model Update*>, (3, 4): <gurobi.Var *Awaiting Model Update*>, (4, 0): <gurobi.Var *Awaiting Model Update*>, (4, 1): <gurobi.Var *Awaiting Mod

In [198]:
model.setObjective(quicksum(d_ij[i,j]*x_ij[i,j] for i in [e[0] for e in S] for j in [e[1] for e in S]),GRB.MINIMIZE); #目的関数の追加

model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [193]:
#最適値の出力
print("Opt.value=",model.objval)

for i in range(5): #最適解の出力
    for j in range(5):
        print("x_ij[",i,"][",j,"]=",x_ij[i,j].X)

AttributeError: Unable to retrieve attribute 'objval'